## Data retrieval and cleaning

In [1]:
from ift6758.data.acquisition import NHLGameData

data_path_raw = './../../ift6758/data/json_raw/'
nhl_games_data = NHLGameData(data_path_raw)
for year in range(2016,2021):
    nhl_games_data.fetch_season(year)



Loading from cache file ./../../ift6758/data/json_raw/2016/2016-regular.pkl
Found 1230 regular games for season 2016-2017
Loading from cache file ./../../ift6758/data/json_raw/2016/2016-playoff.pkl
Found 102 playoff games for season 2016-2017
Loading from cache file ./../../ift6758/data/json_raw/2017/2017-regular.pkl
Found 1271 regular games for season 2017-2018
Loading from cache file ./../../ift6758/data/json_raw/2017/2017-playoff.pkl
Found 105 playoff games for season 2017-2018
Loading from cache file ./../../ift6758/data/json_raw/2018/2018-regular.pkl
Found 1271 regular games for season 2018-2019
Loading from cache file ./../../ift6758/data/json_raw/2018/2018-playoff.pkl
Found 105 playoff games for season 2018-2019
Loading from cache file ./../../ift6758/data/json_raw/2019/2019-regular.pkl
Found 1271 regular games for season 2019-2020
Loading from cache file ./../../ift6758/data/json_raw/2019/2019-playoff.pkl
Found 105 playoff games for season 2019-2020
Loading from cache file ./..

In [2]:
from ift6758.data.cleaning import DataCleaner

data_path_clean = './../../ift6758/data/json_clean/'
data_cleaner = DataCleaner(data_raw=nhl_games_data, data_path_clean=data_path_clean)
for year in range(2016,2021):
    data_cleaner.clean_season(year, keepPreviousEventInfo=True, includePowerPlay=False)


In [3]:
data_2018 = data_cleaner.get_cleaned_data(2018)

In [4]:
data_2018.head()

,game_id,period,period_time,type,team,x,y,shooter,goalie,shot_type,empty_net,strength,opposite_team_side,prev_type,prev_x,prev_y,time_since_prev,distance_from_prev
0,2018020001,1,29,SHOT,Montréal Canadiens,78.0,-19.0,Artturi Lehkonen,Frederik Andersen,Backhand,False,None,right,FACEOFF,0.0,0.0,29.0,80.28
1,2018020001,1,49,SHOT,Toronto Maple Leafs,-37.0,-10.0,Morgan Rielly,Carey Price,Snap Shot,False,None,left,SHOT,78.0,-19.0,20.0,115.35
2,2018020001,1,60,SHOT,Montréal Canadiens,47.0,-23.0,Brendan Gallagher,Frederik Andersen,Wrist Shot,False,None,right,SHOT,-37.0,-10.0,11.0,85.00
3,2018020001,1,234,SHOT,Montréal Canadiens,53.0,14.0,Brendan Gallagher,Frederik Andersen,Snap Shot,False,None,right,MISSED_SHOT,73.0,22.0,5.0,21.54
4,2018020001,1,246,SHOT,Montréal Canadiens,77.0,-2.0,Mike Reilly,Frederik Andersen,Snap Shot,False,None,right,MISSED_SHOT,81.0,-4.0,5.0,4.47


In [5]:
data_2018.isna().sum()

game_id                   0
period                    0
period_time               0
type                      0
team                      0
x                         0
y                         0
shooter                   0
goalie                  420
shot_type                 0
empty_net                 0
strength              75063
opposite_team_side        0
prev_type                 0
prev_x                    0
prev_y                    0
time_since_prev           0
distance_from_prev        0
dtype: int64

## Feature engineering

In [10]:
from ift6758.features import FeatureEng
data_path_clean = './../../ift6758/data/json_clean/'
w = FeatureEng(data_path_clean)

In [11]:
df = w.features_2(2016,2020)
df.sample(10)

,game_id,period,game_seconds,x,y,shot_type,empty_net,prev_type,prev_x,prev_y,time_since_prev,distance_from_prev,distance_goal,prev_distance_goal,angle_shot,prev_angle_shot,bounce,angle_change,speed,is_goal
102736,2017020445,3,4570,0.0,-34.0,Wrist Shot,False,FACEOFF,0.0,0.0,36.0,34.00,96.21,90.00,-20.70,0.00,False,0.00,0.94,0
20202,2016020334,1,2213,81.0,16.0,Snap Shot,False,SHOT,-64.0,-10.0,9.0,147.31,18.36,154.32,60.63,-3.72,True,64.35,16.37,0
234239,2019020036,1,1845,-77.0,10.0,Slap Shot,False,FACEOFF,-20.0,-22.0,17.0,65.37,16.40,73.38,37.57,-17.45,False,0.00,3.85,0
97822,2017020366,1,1467,-79.0,4.0,Deflected,False,GIVEAWAY,-64.0,38.0,9.0,37.16,11.70,46.04,19.99,55.63,False,0.00,4.13,0
157503,2018020053,3,3809,74.0,-5.0,Backhand,False,TAKEAWAY,-84.0,-26.0,30.0,159.39,16.76,175.93,-17.36,-8.50,False,0.00,5.31,0
180547,2018020426,5,6000,-65.0,5.0,Wrist Shot,False,SHOT,69.0,1.0,1.0,134.06,25.50,159.00,11.31,0.36,True,10.95,134.06,1
78743,2017020058,2,3073,-48.0,-23.0,Snap Shot,False,HIT,47.0,-37.0,34.0,96.03,47.89,141.91,-28.70,-15.11,False,0.00,2.82,0
69852,2016021147,1,1332,-87.0,-21.0,Wrist Shot,False,MISSED_SHOT,-65.0,15.0,12.0,42.19,21.21,29.15,-81.93,30.97,True,-112.90,3.52,0
17311,2016020287,2,2668,-44.0,21.0,Wrist Shot,False,FACEOFF,20.0,-22.0,36.0,77.10,135.64,73.38,8.91,-17.45,False,0.00,2.14,0
96995,2017020353,1,2072,-75.0,8.0,Wrist Shot,False,MISSED_SHOT,-60.0,20.0,52.0,19.21,17.00,36.06,28.07,33.69,True,-5.62,0.37,0


In [12]:
df.isna().sum()

game_id               0
period                0
game_seconds          0
x                     0
y                     0
shot_type             0
empty_net             0
prev_type             0
prev_x                0
prev_y                0
time_since_prev       0
distance_from_prev    0
distance_goal         0
prev_distance_goal    0
angle_shot            0
prev_angle_shot       0
bounce                0
angle_change          0
speed                 0
is_goal               0
dtype: int64

In [13]:
df.dtypes

game_id                 int32
period                  int64
game_seconds            int64
x                     float64
y                     float64
shot_type              object
empty_net                bool
prev_type              object
prev_x                float64
prev_y                float64
time_since_prev       float64
distance_from_prev    float64
distance_goal         float64
prev_distance_goal    float64
angle_shot            float64
prev_angle_shot       float64
bounce                   bool
angle_change          float64
speed                 float64
is_goal                 int32
dtype: object

In [14]:
df = w.encodeCategories(df, ['shot_type', 'prev_type'])

In [15]:
df = df.rename(columns = {'shot_type_Slap Shot':'shot_type_Slap_Shot', 'shot_type_Snap Shot':'shot_type_Snap_Shot', 'shot_type_Wrist Shot':'shot_type_Wrist_Shot'})


In [16]:
df = df.drop(['game_id'], axis = 1)
df.dtypes

period                     int64
game_seconds               int64
x                        float64
y                        float64
empty_net                   bool
prev_x                   float64
prev_y                   float64
time_since_prev          float64
distance_from_prev       float64
distance_goal            float64
prev_distance_goal       float64
angle_shot               float64
prev_angle_shot          float64
bounce                      bool
angle_change             float64
speed                    float64
is_goal                    int32
shot_type_Deflected         bool
shot_type_Slap_Shot         bool
shot_type_Snap_Shot         bool
shot_type_Tip-In            bool
shot_type_Wrap-around       bool
shot_type_Wrist_Shot        bool
prev_type_FACEOFF           bool
prev_type_GIVEAWAY          bool
prev_type_GOAL              bool
prev_type_HIT               bool
prev_type_MISSED_SHOT       bool
prev_type_PENALTY           bool
prev_type_SHOT              bool
prev_type_

In [17]:
df.to_pickle('./TrainValSets2.pkl')

## Log data

We log game "Winnipeg vs Washington" the 12th march 2018 (game 2017021065) into Comet.ml

In [ ]:
game_df = df[df.game_id == 2017021065]
game_df.head()

In [ ]:
import os
from comet_ml import Experiment

exp = Experiment(
        api_key=os.environ.get('COMET_API_KEY'),
        workspace='ift6758-a5-nhl',
        project_name='milestone2'
    )

exp.log_dataframe_profile(
    game_df,
    name='wpg_v_wsh_2017021065',
    dataframe_format='csv',
    )

exp.end()